---
description: API details.
output-file: machinery_calculation.html
title: module name here

---



In [ ]:
import os
import random

import numpy as np
import pandas as pd
from feems.components_model.component_electric import Genset
from feems.components_model.component_mechanical import (
    EngineMultiFuel,
    FuelCharacteristics,
)
from feems.fuel import FuelOrigin, FuelSpecifiedBy, TypeFuel
from feems.system_model import FuelOption
from feems.types_for_feems import EngineCycleType, TypeComponent
from MachSysS.convert_to_feems import convert_proto_propulsion_system_to_feems
from MachSysS.gymir_result_pb2 import (
    GymirResult,
    SimulationInstance,
    TimeSeriesResult,
)
from MachSysS.utility import retrieve_machinery_system_from_file
from RunFeemsSim.machinery_calculation import (
    MachineryCalculation,
    convert_gymir_result_to_propulsion_power_series,
)

pd.options.plotting.backend = "plotly"

In [ ]:
def create_gymir_result() -> GymirResult:
    return GymirResult(
        name="test",
        auxiliary_load_kw=random.random() * 500,
        result=[
            SimulationInstance(epoch_s=100 * index + 1, power_kw=2000 * random.random())
            for index, _ in enumerate(range(10))
        ],
    )


# Testing convert gymir result to propulsion power series
gymir_result = create_gymir_result()
propulsion_power = convert_gymir_result_to_propulsion_power_series(gymir_result)
assert len(gymir_result.result) == len(propulsion_power)
propulsion_power.plot()

# Testing MachineryCalculation
save_the_result = False
path = os.path.join("tests", "system_proto.mss")
system_proto = retrieve_machinery_system_from_file(path)
print(system_proto)
system_feems = convert_proto_propulsion_system_to_feems(system_proto)
machinery_calculation = MachineryCalculation(feems_system=system_feems)
res = machinery_calculation.calculate_machinery_system_output_from_gymir_result(
    gymir_result=gymir_result
)
if save_the_result:
    with open("system_proto.mss", "wb") as file:
        file.write(system_proto.SerializeToString())
average_power_consumption = 0
running_hours_mean = 0
for power_source in machinery_calculation.electric_system.power_sources:
    running_hours_mean += (power_source.power_output > 0).mean()
for propulsor in machinery_calculation.electric_system.propulsion_drives:
    average_power_consumption += propulsor.power_input.mean()
average_bsfc_for_engines = res.fuel_consumption_total_kg / (
    res.duration_s / 3600000 * average_power_consumption / 0.95
)
print(f"Average bsfc for engines: {average_bsfc_for_engines} g/kWh")

name: "Electric propulsion system"
propulsion_type: ELECTRIC
maximum_allowed_genset_load_percentage: 80
electric_system {
  switchboards {
    switchboard_id: 1
    subsystems {
      engine {
        rated_power_kw: 1000
        rated_speed_rpm: 1000
        bsfc {
          curve {
            x_label: "power load"
            y_label: "bsfc"
            curve {
              points {
                x: 0.1
                y: 270
              }
              points {
                x: 0.25
                y: 240
              }
              points {
                x: 0.5
                y: 220
              }
              points {
                x: 0.75
                y: 210
              }
              points {
                x: 1
                y: 220
              }
            }
          }
        }
        main_fuel {
          fuel_origin: FOSSIL
        }
        order_from_switchboard_or_shaftline: 2
        nox_calculation_method: TIER_3
        engine_cycle_type:

In [ ]:
# Testing MachineryCalculation with time series result (protobuf)
path_to_system = os.path.join("tests", "mechanical_propulsion_with_electric_system.mss")
system_proto = retrieve_machinery_system_from_file(path_to_system)
system_feems = convert_proto_propulsion_system_to_feems(system_proto)

path_to_result = os.path.join("tests", "time_series_result.pb")
with open(path_to_result, "rb") as file:
    time_series_result = TimeSeriesResult()
    time_series_result.ParseFromString(file.read())

machinery_calculation = MachineryCalculation(feems_system=system_feems)
res = machinery_calculation.calculate_machinery_system_output_from_time_series_result(
    time_series=time_series_result
)

In [ ]:
# Testing MachineryCalculation with time series result as dataframe (protobuf)
path_to_system = os.path.join("tests", "mechanical_propulsion_with_electric_system.mss")
system_proto = retrieve_machinery_system_from_file(path_to_system)
system_feems = convert_proto_propulsion_system_to_feems(system_proto)

path_to_result = os.path.join("tests", "time_series_result.pb")
with open(path_to_result, "rb") as file:
    time_series_result = TimeSeriesResult()
    time_series_result.ParseFromString(file.read())


machinery_calculation = MachineryCalculation(feems_system=system_feems)
res = machinery_calculation.calculate_machinery_system_output_from_time_series_result(
    time_series=time_series_result
)

In [ ]:
# Create a multi-fuel engine for the main engine
# Testing MachineryCalculation with time series result as dataframe (protobuf)
path_to_system = os.path.join("tests", "mechanical_propulsion_with_electric_system.mss")
system_proto = retrieve_machinery_system_from_file(path_to_system)
system_feems = convert_proto_propulsion_system_to_feems(system_proto)
main_engine = system_feems.mechanical_system.main_engines[0]
bsfc_vlsfo = main_engine.engine.specific_fuel_consumption_points.copy()
bsfc_vlsfo[:, 1] *= 48 / 41  # Assume 41.7 MJ/kg for VLSFO
multi_fuel_main_engine = EngineMultiFuel(
    type_=TypeComponent.MAIN_ENGINE,
    name=main_engine.engine.name,
    rated_power=main_engine.engine.rated_power,
    rated_speed=main_engine.engine.rated_speed,
    multi_fuel_characteristics=[
        FuelCharacteristics(
            main_fuel_type=TypeFuel.NATURAL_GAS,
            main_fuel_origin=FuelOrigin.FOSSIL,
            pilot_fuel_type=TypeFuel.DIESEL,
            pilot_fuel_origin=FuelOrigin.FOSSIL,
            bsfc_curve=main_engine.engine.specific_fuel_consumption_points,
            bspfc_curve=main_engine.engine.specific_pilot_fuel_consumption_points,
            engine_cycle_type=EngineCycleType.OTTO,
        ),
        FuelCharacteristics(
            main_fuel_type=TypeFuel.VLSFO,
            main_fuel_origin=FuelOrigin.FOSSIL,
            bsfc_curve=bsfc_vlsfo,
            engine_cycle_type=EngineCycleType.OTTO,
        ),
    ],
)
main_engine.engine = multi_fuel_main_engine

multi_fuel_gensets = []
aux_engine = system_feems.electric_system.power_sources[0].aux_engine
bspfc_curve = np.array([[0, 0], [0.25, 1.7], [0.5, 1.2], [0.75, 0.8], [1.0, 0.6]])
bsfc_vlsfo = aux_engine.specific_fuel_consumption_points.copy()
bsfc_vlsfo[:, 1] *= 48 / 41
for power_source in system_feems.electric_system.power_sources:
    if power_source.type == TypeComponent.GENSET:
        genset: Genset = power_source  # type: ignore
        multi_fuel_genset = EngineMultiFuel(
            type_=TypeComponent.AUXILIARY_ENGINE,
            name=genset.name,
            rated_power=genset.aux_engine.rated_power,
            rated_speed=genset.aux_engine.rated_speed,
            multi_fuel_characteristics=[
                FuelCharacteristics(
                    main_fuel_type=TypeFuel.NATURAL_GAS,
                    main_fuel_origin=FuelOrigin.FOSSIL,
                    pilot_fuel_type=TypeFuel.DIESEL,
                    pilot_fuel_origin=FuelOrigin.FOSSIL,
                    bsfc_curve=genset.aux_engine.specific_fuel_consumption_points,
                    bspfc_curve=bspfc_curve,
                    engine_cycle_type=EngineCycleType.OTTO,
                ),
                FuelCharacteristics(
                    main_fuel_type=TypeFuel.VLSFO,
                    main_fuel_origin=FuelOrigin.FOSSIL,
                    bsfc_curve=bsfc_vlsfo,
                    engine_cycle_type=EngineCycleType.DIESEL,
                ),
            ],
        )
        genset.aux_engine = multi_fuel_genset


# Test the fuel option argument in the methods of MachineryCalculation
machinery_calculation = MachineryCalculation(feems_system=system_feems)
res_no_option_imo = (
    machinery_calculation.calculate_machinery_system_output_from_time_series_result(
        time_series=time_series_result,
        fuel_specified_by=FuelSpecifiedBy.IMO,
    )
)

res_VLSFO_imo = (
    machinery_calculation.calculate_machinery_system_output_from_time_series_result(
        time_series=time_series_result,
        fuel_specified_by=FuelSpecifiedBy.IMO,
        fuel_option=FuelOption(fuel_type=TypeFuel.VLSFO, fuel_origin=FuelOrigin.FOSSIL, for_pilot=False, primary=False),
    )
)

res_NATURAL_GAS_imo = (
    machinery_calculation.calculate_machinery_system_output_from_time_series_result(
        time_series=time_series_result,
        fuel_specified_by=FuelSpecifiedBy.IMO,
        fuel_option=FuelOption(
            fuel_type=TypeFuel.NATURAL_GAS, fuel_origin=FuelOrigin.FOSSIL, for_pilot=False, primary=True,
        ),
    )
)

In [ ]:
print("Total fuel energy consumption for mechanical system in tons:")
total_energy_consumption_default_gj = sum(
    [
        fuel.mass_or_mass_fraction * fuel.lhv_mj_per_g
        for fuel in res_no_option_imo.mechanical_system.multi_fuel_consumption_total_kg.fuels
    ]
)
total_energy_consumption_vlsfo_gj = sum(
    [
        fuel.mass_or_mass_fraction * fuel.lhv_mj_per_g
        for fuel in res_VLSFO_imo.mechanical_system.multi_fuel_consumption_total_kg.fuels
    ]
)
total_energy_consumption_natural_gas_gj = sum(
    [
        fuel.mass_or_mass_fraction * fuel.lhv_mj_per_g
        for fuel in res_NATURAL_GAS_imo.mechanical_system.multi_fuel_consumption_total_kg.fuels
    ]
)
print(f" - Default Fuel (LNG) : {total_energy_consumption_default_gj / 1000:.2f}")
print(f" - VLSFO : {total_energy_consumption_vlsfo_gj / 1000:.2f}")
print(f" - Natural Gas : {total_energy_consumption_natural_gas_gj / 1000:.2f}")
assert (
    total_energy_consumption_default_gj == total_energy_consumption_natural_gas_gj
), "Total energy consumption should be the same for different fuel options."
assert (
    abs(total_energy_consumption_default_gj - total_energy_consumption_vlsfo_gj)
    / total_energy_consumption_default_gj
    < 1e-2
), "Total energy consumption should be the same for different fuel options."
print("Total fuel consumption for electric system in tons:")
total_energy_consumption_default_gj = sum(
    [
        fuel.mass_or_mass_fraction * fuel.lhv_mj_per_g
        for fuel in res_no_option_imo.electric_system.multi_fuel_consumption_total_kg.fuels
    ]
)
total_energy_consumption_vlsfo_gj = sum(
    [
        fuel.mass_or_mass_fraction * fuel.lhv_mj_per_g
        for fuel in res_VLSFO_imo.electric_system.multi_fuel_consumption_total_kg.fuels
    ]
)
total_energy_consumption_natural_gas_gj = sum(
    [
        fuel.mass_or_mass_fraction * fuel.lhv_mj_per_g
        for fuel in res_NATURAL_GAS_imo.electric_system.multi_fuel_consumption_total_kg.fuels
    ]
)
print(f" - Default Fuel (LNG) : {total_energy_consumption_default_gj / 1000:.2f}")
print(f" - VLSFO : {total_energy_consumption_vlsfo_gj / 1000:.2f}")
print(f" - Natural Gas : {total_energy_consumption_natural_gas_gj / 1000:.2f}")
assert (
    total_energy_consumption_default_gj == total_energy_consumption_natural_gas_gj
), "Total energy consumption should be the same for different fuel options."
assert (
    abs(total_energy_consumption_default_gj - total_energy_consumption_vlsfo_gj)
    / total_energy_consumption_default_gj
    < 1e-2
), "Total energy consumption should be the same for different fuel options."
print("GHG Emissions in for mechanical system tons:")
print(
    f" - Default Fuel (LNG) : {res_no_option_imo.mechanical_system.co2_emission_total_kg.tank_to_wake_kg_or_gco2eq_per_gfuel / 1000:.2f}"
)
print(
    f" - VLSFO : {res_VLSFO_imo.mechanical_system.co2_emission_total_kg.tank_to_wake_kg_or_gco2eq_per_gfuel / 1000:.2f}"
)
print(
    f" - Natural Gas : {res_NATURAL_GAS_imo.mechanical_system.co2_emission_total_kg.tank_to_wake_kg_or_gco2eq_per_gfuel / 1000:.2f}"
)

print("GHG Emissions in for electric system tons:")
print(
    f" - Default Fuel (LNG) : {res_no_option_imo.electric_system.co2_emission_total_kg.tank_to_wake_kg_or_gco2eq_per_gfuel / 1000:.2f}"
)
print(
    f" - VLSFO : {res_VLSFO_imo.electric_system.co2_emission_total_kg.tank_to_wake_kg_or_gco2eq_per_gfuel / 1000:.2f}"
)
print(
    f" - Natural Gas : {res_NATURAL_GAS_imo.electric_system.co2_emission_total_kg.tank_to_wake_kg_or_gco2eq_per_gfuel / 1000:.2f}"
)

Total fuel energy consumption for mechanical system in tons:
 - Default Fuel (LNG) : 144.89
 - VLSFO : 144.10
 - Natural Gas : 144.89
Total fuel consumption for electric system in tons:
 - Default Fuel (LNG) : 27.05
 - VLSFO : 26.91
 - Natural Gas : 27.05
GHG Emissions in for mechanical system tons:
 - Default Fuel (LNG) : 8315.19
 - VLSFO : 11267.58
 - Natural Gas : 8315.19
GHG Emissions in for electric system tons:
 - Default Fuel (LNG) : 1552.26
 - VLSFO : 2104.61
 - Natural Gas : 1552.26
